<a href="https://colab.research.google.com/github/MateoOrtiz001/TareasML/blob/main/Experimentos%20Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Code Introduction**


This code was extracted from: https://github.com/Vasanthengineer4949/NLP-Projects-NHV, and is explained in the YouTube video: https://www.youtube.com/watch?v=crXjRYbvT1U. It has been slightly modified to include an inference without DPO. It is presented as supplementary material for the "Human Feedback for LLMs" project, which is the final project for the course Mathematics for Machine Learning 2024-1. This code serves as an important resource to learn the process of fine-tuning an LLM using the DPO algorithm.

The following code is a guide to fine-tune a pre-trained model. The model can be found at: https://huggingface.co/TheBloke/OpenHermes-2-Mistral-7B-GPTQ. It has been adjusted to human preferences using a "prompt-chosen-rejected" format dataset, which was sourced from: https://huggingface.co/datasets/HuggingFaceH4/ultrafeedback_binarized. This dataset is called "ultrafeedback" because LLMs were used to generate the dataset and choose the rejected and selected responses. This dataset, and not a "human feedback" type, was used because they have the same format, and the purpose of this code is to illustrate the process of adjusting an LLM using the DPO algorithm. The ultrafeedback dataset can be replaced with human feedback without any issues.

Several libraries are used, such as "torch" and "transformers" to manipulate the models, "peft" to perform "parameter efficient fine-tuning", "trl" to train with DPO, and others like "bitsandbytes", "optimum", "auto-gpt" to quantize the model, reducing its memory requirements for training and execution.

The code utilizes the T4 GPU provided by Colab and takes a few minutes to execute. It is necessary to have a Hugging Face account and authenticate it.

The process of the code is as follows:

* Extract the "OpenHermes-2-Mistral-7B-GPTQ" model from HuggingFace, along with the "ultrafeedback_binarized" dataset.
* Create a copy of the model, a reference model, necessary for the DPO algorithm.
* Adapt the models to PEFT, adjust the training arguments, and train the DPO with both models.
* Reveal certain metrics such as training loss, validation loss, among others.
* Finally, upload the trained model to our Hugging Face account, extract this model, and perform an inference with it.
* Compare this inference with that of the original model, the one to which DPO was not applied.

##Preliminaries and Libraries

In [ ]:
#Hugging Face Authentication
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
! pip install torch==2.0.1 transformers datasets peft accelerate trl bitsandbytes optimum auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.4/418.4 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 817.6 kB/s eta 0:00:00
     ━━━━━━

In [ ]:
import torch
from datasets import Dataset, load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM, GPTQConfig
from trl import DPOTrainer

##Model and Training Data Extraction

In [ ]:
#function used to adapt the ultrafeedback_binerized dataset to one with the "prompt-chosen-rejected" format.
def dpo_data():

    dataset = load_dataset(
        "HuggingFaceH4/ultrafeedback_binarized",
        split = "test_prefs",
        use_auth_token=True
    )

    original_columns = dataset.column_names

    def return_prompt_and_responses(samples):
        return {
            "prompt": [prompt for prompt in samples["prompt"]],
            "chosen": samples["chosen"],
            "rejected": samples["rejected"],
        }

    return dataset.map(
        return_prompt_and_responses,
        batched=True,
        remove_columns=original_columns,
    )

In [ ]:
#extract a tokenizer from the "OpenHermes-2-Mistral-7B-GPTQ" model from HuggingFace.
tokenizer = AutoTokenizer.from_pretrained("TheBloke/OpenHermes-2-Mistral-7B-GPTQ")

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Extract the "OpenHermes-2-Mistral-7B-GPTQ" model and a copy of it as a reference model, which will later be used in the DPO algorithm
# Minimize memory usage as much as possible, for example, by using only 4 bits to represent weights and activations.
model = AutoModelForCausalLM.from_pretrained("TheBloke/OpenHermes-2-Mistral-7B-GPTQ", torch_dtype=torch.float16, low_cpu_mem_usage=True, quantization_config=GPTQConfig(bits=4, disable_exllama=True))

model_ref = AutoModelForCausalLM.from_pretrained("TheBloke/OpenHermes-2-Mistral-7B-GPTQ", torch_dtype=torch.float16, low_cpu_mem_usage=True, quantization_config=GPTQConfig(bits=4, disable_exllama=True))

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/OpenHermes-2-Mistral-7B-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.lay

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you p

In [ ]:
train_dataset = dpo_data()

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2547: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Generating train_prefs split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating train_sft split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_prefs split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 2000
})

In [ ]:
train_df = train_dataset.to_pandas()
train_df

,prompt,chosen,rejected
0,"In this task, you are given a second sentence....","[{'content': 'In this task, you are given a se...","[{'content': 'In this task, you are given a se..."
1,The floor of a rectangular room is 19 m long a...,[{'content': 'The floor of a rectangular room ...,[{'content': 'The floor of a rectangular room ...
2,"Definition: In this task, you are given an abs...","[{'content': 'Definition: In this task, you ar...","[{'content': 'Definition: In this task, you ar..."
3,Evaluate the extent to which web usability is ...,[{'content': 'Evaluate the extent to which web...,[{'content': 'Evaluate the extent to which web...
4,A text is given in Bengali. Translate it from ...,[{'content': 'A text is given in Bengali. Tran...,[{'content': 'A text is given in Bengali. Tran...
...,...,...,...
1995,can you give me an overview of my mri medical ...,[{'content': 'can you give me an overview of m...,[{'content': 'can you give me an overview of m...
1996,"QUESTION: Can we conclude from ""Two men hold b...","[{'content': 'QUESTION: Can we conclude from ""...","[{'content': 'QUESTION: Can we conclude from ""..."
1997,Construct lyrics in the style of The Proclaime...,[{'content': 'Construct lyrics in the style of...,[{'content': 'Construct lyrics in the style of...
1998,"Detailed Instructions: In this task, you will ...",[{'content': 'Detailed Instructions: In this t...,[{'content': 'Detailed Instructions: In this t...


In [ ]:
train_df["chosen"] = train_df["chosen"].apply(lambda x: x[1]["content"])
train_df["rejected"] = train_df["rejected"].apply(lambda x: x[1]["content"])

In [ ]:
train_df = train_df.dropna()

In [ ]:
val_df = train_df.sample(10)

In [ ]:
train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)

In [ ]:
train_data

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 2000
})

##Model Definition and PEFT Adaptation

In [ ]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (rotary_emb): MistralRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32002, bias=False)
)

In [ ]:
# We adjust the PEFT configuration, which is necessary for efficiently fine-tuning
# the model by adjusting a portion of the parameters rather than all of them.
peft_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=["q_proj", "v_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )
peft_config.inference_mode = False

In [ ]:
# The model is adapted to PEFT
model = prepare_model_for_kbit_training(model)
model.config.use_cache=False
model.gradient_checkpointing_enable()
model.config.pretraining_tp=1
model = get_peft_model(model, peft_config)

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32002, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (rotary_emb): MistralRotaryEmbedding()
              (k_proj): QuantLinear()
              (o_proj): QuantLinear()
              (q_proj): lora.QuantLinear(
                (base_layer): QuantLinear()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterD

In [ ]:
# The same is done with the reference model, adapting it to PEFT.
model_ref = prepare_model_for_kbit_training(model_ref)
model_ref.config.use_cache=False
model_ref.gradient_checkpointing_enable()
model_ref.config.pretraining_tp=1
model_ref = get_peft_model(model_ref, peft_config)

##Model Training

In [ ]:
# We set up the training arguments to be used in the DPO trainer.
training_args = TrainingArguments(
        per_device_train_batch_size=1,
        max_steps=50,
        remove_unused_columns=False,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        evaluation_strategy="steps",
        logging_first_step=True,
        logging_steps=10,
        output_dir="openhermes-mistral-dpo-gptq",
        optim="paged_adamw_32bit",
        warmup_steps=2,
        fp16=True,
        push_to_hub=True
    )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
dpo_trainer = DPOTrainer(
        model,
        model_ref,
        args=training_args,
        beta=0.1,
        train_dataset=train_data,
        eval_dataset=val_data,
        tokenizer=tokenizer,
        max_length=512,
        max_target_length=256,
        max_prompt_length=256
    )

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# We train the model using the DPO algorithm, revealing some metrics.
dpo_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
10,0.676800,0.696490,0.020577,-0.007018,0.562500,0.027595,-168.454819,-127.324234,-2.400090,-2.316475
20,0.706200,0.713050,0.004261,0.042102,0.250000,-0.037841,-167.963608,-127.487396,-2.407761,-2.315611
30,0.752600,0.710098,-0.004784,0.047834,0.125000,-0.052618,-167.906296,-127.577850,-2.408823,-2.312083
40,0.694600,0.700297,-0.014361,0.018028,0.187500,-0.032389,-168.204361,-127.673615,-2.408658,-2.309932
50,0.672800,0.686409,-0.017557,-0.007723,0.312500,-0.009834,-168.461868,-127.705566,-2.407153,-2.307698


TrainOutput(global_step=50, training_loss=0.7009437656402588, metrics={'train_runtime': 365.7166, 'train_samples_per_second': 0.137, 'train_steps_per_second': 0.137, 'total_flos': 0.0, 'train_loss': 0.7009437656402588, 'epoch': 0.025})

In [ ]:
# We push the model to our HuggingFace account
dpo_trainer.push_to_hub("19Leo97/openhermes-mistral-dpo-gptq")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

events.out.tfevents.1717099527.bdd30f5aa120.706.0:   0%|          | 0.00/13.6k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/19Leo97/openhermes-mistral-dpo-gptq/commit/a55c758d2adee6c812f9908431b8d1074cb3cdfd', commit_message='19Leo97/openhermes-mistral-dpo-gptq', commit_description='', oid='a55c758d2adee6c812f9908431b8d1074cb3cdfd', pr_url=None, pr_revision=None, pr_num=None)

##Model Inference with and without DPO

### Inference without DPO applied

In [ ]:
# We extract the model without DPO applied, its tokenizer, and adjust the necessary configurations for inference.
# We define our input sentence

from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("TheBloke/OpenHermes-2-Mistral-7B-GPTQ")

inputs = tokenizer("""I have dropped my phone in water. Now it is not working what should I do now?""", return_tensors="pt").to("cuda")

model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/OpenHermes-2-Mistral-7B-GPTQ",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/OpenHermes-2-Mistral-7B-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.lay

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [ ]:
# We perform inference
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

I have dropped my phone in water. Now it is not working what should I do now?

If you have dropped your phone in water, the first thing you should do is to turn it off immediately. If it is still on, turn it off. Then remove the battery if possible. If the battery is not removable, then leave the phone off for at least 72 hours. After that, try to turn it on. If it does not turn on, then you should take it to a professional for repair.

What should I do if my phone is not charging?

If your phone is not charging, first check the charger. If the charger is working fine, then check the phone. If the phone is not charging, then you should take it to a professional for repair.

What should I do if my phone is not receiving calls or messages?

If your phone is not receiving calls or messages, first check the network signal. If the network signal is weak, then try to move to a place with better network coverage. If the network signal is strong, then check the phone settings. If the phone set

### Inference with DPO applied

In [ ]:
# We extract the model with DPO applied, its tokenizer, and adjust the necessary configurations for inference.
# We define our input sentence

from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("19Leo97/openhermes-mistral-dpo-gptq")

inputs = tokenizer("""I have dropped my phone in water. Now it is not working what should I do now?""", return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM.from_pretrained(
    "19Leo97/openhermes-mistral-dpo-gptq",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/OpenHermes-2-Mistral-7B-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layer

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [ ]:
# We perform inference
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

I have dropped my phone in water. Now it is not working what should I do now?

If you have dropped your phone in water, the first thing you should do is to turn it off immediately. If it is still on, turn it off. Then remove the battery if possible. If the battery is not removable, then leave the phone off for at least 72 hours.

After 72 hours, try to turn the phone on. If it does not turn on, then you may have to take it to a professional for repair.

If the phone turns on, then you should check the phone for any water damage. If there is water damage, then you may have to take it to a professional for repair.

If there is no water damage, then you should check the phone for any other damage. If there is no other damage, then you should be able to use the phone as normal. source: http://www.wikihow.com/Fix-a-Water-Damaged-Phone source: http://www.wikihow.com/Fix-a-Water-Damaged-Phone source: http://www.wikihow.com/Fix-a-Water-Damaged-Phone source: http://www.wikihow.com
268.304576396